# Llama-3 8B (QLoRA ile İnce Ayarlanmış) Finansal Duygu Analizi Test Notebook'u

Bu notebook, `azizdeniz890/Llama3-8B-Financial-Sentiment-LoRA` Hugging Face Hub reposundaki QLoRA adaptörleri ile ince ayarlanmış `meta-llama/Meta-Llama-3-8B-Instruct` modelini kullanarak finansal metinler üzerinde duygu analizi (pozitif, negatif, nötr) yapmak için hazırlanmıştır.

## Amaç

Bu notebook'un temel amacı:

1.  Gerekli kütüphaneleri yüklemek.
2.  Temel Llama-3 modelini 4-bit kuantizasyon ile yüklemek.
3.  Belirtilen Hugging Face Hub reposundan eğitilmiş QLoRA adaptörlerini ve tokenizer'ı yüklemek.
4.  Modeli bu adaptörlerle birleştirmek.
5.  Örnek finansal cümleler üzerinde duygu analizi tahminleri yapmak ve sonuçları göstermek.

## Kullanım

1.  **Ortam:** Bu notebook Google Colab ortamında (tercihen A100 GPU ile) çalıştırılmak üzere tasarlanmıştır.
2.  **Hugging Face Token:** `meta-llama/Meta-Llama-3-8B-Instruct` modeline veya `azizdeniz890/Llama3-8B-Financial-Sentiment-LoRA` reposuna erişim için Hugging Face token'ınız gerekebilir. İkinci hücredeki `notebook_login()` adımı sizden token isteyecektir. (Eğer Colab Secrets kullanılıyorsa, `HF_TOKEN` adında bir secret oluşturulmalıdır).
3.  **Çalıştırma:** Hücreleri yukarıdan aşağıya doğru sırasıyla çalıştırın.
    *   İlk hücre kütüphane kurulumlarını yapar.
    *   Sonraki hücreler modeli, tokenizer'ı ve adaptörleri yükler.
    *   Son hücreler ise örnek cümleler üzerinde tahminler yapar.

## Model ve Adaptör Bilgileri

*   **Temel Model:** `meta-llama/Meta-Llama-3-8B-Instruct`
*   **Adaptör Reposu (Hugging Face Hub):** `azizdeniz890/Llama3-8B-Financial-Sentiment-LoRA`

Bu notebook, eğitilmiş modelin hızlıca test edilmesini ve performansının gözlemlenmesini sağlamak için basit bir arayüz sunar.

In [1]:
# --- Gerekli Kütüphaneleri Yükle ---
print("Gerekli kütüphaneler yükleniyor...")
# PyTorch'u CUDA 12.1 uyumlu versiyonuyla yükleyelim (A100 genellikle bunu destekler)
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
# bitsandbytes'ı özellikle ve belirli bir versiyon üzeri olarak yükle
!pip install -q bitsandbytes>=0.39.0
!pip install -q transformers>=4.38.0 peft>=0.9.0 accelerate>=0.25.0 huggingface_hub
print("Kütüphane yüklemesi tamamlandı.")

# --- Temel Kütüphaneleri Import Et ---
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
from peft import PeftModel
from huggingface_hub import notebook_login
import os # os modülünü de import edelim, gerekmese bile iyi bir alışkanlık

# --- GPU Kullanılabilirliğini ve Compute Dtype'ı Belirle ---
if torch.cuda.is_available():
    device_global = torch.device("cuda")
    gpu_name = torch.cuda.get_device_name(0)
    print(f"\\nGPU kullanılabilir: {gpu_name}")
    # A100 bfloat16'yı destekler, ancak eğitimde float16 kullanıldığı için
    # çıkarımda da float16 ile devam etmek genellikle en tutarlısıdır.
    compute_dtype_global = getattr(torch, "float16")
else:
    device_global = torch.device("cpu")
    compute_dtype_global = getattr(torch, "float32") # CPU için float32
    print("\\nGPU bulunamadı, CPU kullanılacak. Bu, işlemi önemli ölçüde yavaşlatabilir.")
print(f"Kullanılacak compute_dtype: {compute_dtype_global}")

# --- Yüklü Kütüphane Versiyonlarını Kontrol Et (Hata Ayıklama İçin) ---
print("\\n--- Yüklü Kütüphane Versiyonları (Kontrol) ---")
print(f"PyTorch versiyonu: {torch.__version__}")
if torch.cuda.is_available():
    print(f"PyTorch CUDA versiyonu: {torch.version.cuda}")
try:
    import bitsandbytes
    print(f"bitsandbytes versiyonu: {bitsandbytes.__version__}")
except ImportError:
    print("HATA: bitsandbytes import edilemedi! Lütfen yukarıdaki pip install komutunu kontrol edin.")
import transformers
print(f"transformers versiyonu: {transformers.__version__}")
import peft
print(f"peft versiyonu: {peft.__version__}")
import accelerate
print(f"accelerate versiyonu: {accelerate.__version__}")

# --- Ortam Değişkenleri (Opsiyonel ama bazen faydalı) ---
os.environ["TOKENIZERS_PARALLELISM"] = "false" # Uyarıları azaltabilir

Gerekli kütüphaneler yükleniyor...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.5/780.5 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 95.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 115.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 12.3 MB/s eta 0:

In [2]:
# --- Model ve Adaptör Kimlikleri ---
base_model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
adapter_repo_id = "azizdeniz890/Llama3-8B-Financial-Sentiment-LoRA" # Sizin Hub'daki repo adınız

print(f"Kullanılacak Temel Model: {base_model_id}")
print(f"Kullanılacak Adaptör Reposu: {adapter_repo_id}")
print("\\nHugging Face Hub'a giriş yapılıyor (gerekirse token istenir)...")
try:

    notebook_login() # Şimdilik doğrudan notebook_login kullanalım.
    print("Hugging Face Hub'a giriş denendi/başarılı.")
except Exception as e:
    print(f"Hugging Face Hub'a giriş yapılamadı: {e}")
    print("Eğer model ve adaptörler tamamen public ise ve Llama-3 erişiminiz varsa bu adım sorun olmayabilir.")

Kullanılacak Temel Model: meta-llama/Meta-Llama-3-8B-Instruct
Kullanılacak Adaptör Reposu: azizdeniz890/Llama3-8B-Financial-Sentiment-LoRA
\nHugging Face Hub'a giriş yapılıyor (gerekirse token istenir)...


Hugging Face Hub'a giriş denendi/başarılı.


In [3]:
# --- 4-bit Kuantizasyon Ayarları ---
# compute_dtype_global değişkeni Hücre 1'de tanımlandı.
print(f"\\nKuantizasyon için kullanılacak compute_dtype: {compute_dtype_global}")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype_global,
    bnb_4bit_use_double_quant=False, # Genellikle False, eğitimdeki config'e göre ayarlayın
)

# --- Temel Modeli Yükle ---
print(f"\\nTemel model '{base_model_id}' yükleniyor (4-bit kuantizasyon ile)...")
try:
    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_id,
        quantization_config=bnb_config,
        torch_dtype=compute_dtype_global, # Yükleme sırasında ağırlıkların olacağı dtype
        device_map="auto" # GPU varsa otomatik olarak GPU'ya yükler
    )
    print("Temel model başarıyla yüklendi.")
except Exception as e:
    print(f"Temel model yüklenirken HATA: {e}")
    print("Lütfen internet bağlantınızı, model ID'sini ve Hugging Face token/erişim durumunu kontrol edin.")
    raise

\nKuantizasyon için kullanılacak compute_dtype: torch.float16
\nTemel model 'meta-llama/Meta-Llama-3-8B-Instruct' yükleniyor (4-bit kuantizasyon ile)...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Temel model başarıyla yüklendi.


In [4]:
# --- Tokenizer'ı Adaptör Reposundan Yükle ---
print(f"\\nTokenizer '{adapter_repo_id}' reposundan yükleniyor...")
try:
    tokenizer = AutoTokenizer.from_pretrained(adapter_repo_id)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        print(f"tokenizer.pad_token, tokenizer.eos_token ('{tokenizer.eos_token}') olarak ayarlandı.")
    print("Tokenizer başarıyla yüklendi.")
    print(f"Tokenizer Pad Token: {tokenizer.pad_token} (ID: {tokenizer.pad_token_id})")
    print(f"Tokenizer EOS Token: {tokenizer.eos_token} (ID: {tokenizer.eos_token_id})")
except Exception as e:
    print(f"Tokenizer yüklenirken HATA: {e}")
    print(f"Lütfen '{adapter_repo_id}' reposunda tokenizer dosyalarının olduğundan emin olun.")
    raise

\nTokenizer 'azizdeniz890/Llama3-8B-Financial-Sentiment-LoRA' reposundan yükleniyor...


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

Tokenizer başarıyla yüklendi.
Tokenizer Pad Token: <|eot_id|> (ID: 128009)
Tokenizer EOS Token: <|eot_id|> (ID: 128009)


In [5]:
# --- LoRA Adaptörlerini Temel Modele Yükle ---
print(f"\\nLoRA adaptörleri '{adapter_repo_id}' reposundan temel modele yükleniyor...")
try:
    model_with_adapters = PeftModel.from_pretrained(base_model, adapter_repo_id)
    model_with_adapters.eval() # Modeli değerlendirme (inference) moduna al
    # Çıkarım için use_cache=True genellikle daha hızlıdır, ancak eğitimde False olmalıdır.
    # Eğer model config'inde varsa, açılabilir.
    if hasattr(model_with_adapters, 'config') and hasattr(model_with_adapters.config, 'use_cache'):
         model_with_adapters.config.use_cache = True
    print("LoRA adaptörleri başarıyla yüklendi ve model çıkarım için hazır.")
except Exception as e:
    print(f"LoRA adaptörleri yüklenirken HATA: {e}")
    print(f"Lütfen '{adapter_repo_id}' reposunda geçerli adaptör dosyalarının olduğundan emin olun.")
    raise

\nLoRA adaptörleri 'azizdeniz890/Llama3-8B-Financial-Sentiment-LoRA' reposundan temel modele yükleniyor...


adapter_config.json:   0%|          | 0.00/955 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/671M [00:00<?, ?B/s]

LoRA adaptörleri başarıyla yüklendi ve model çıkarım için hazır.


In [11]:
# --- Tahmin için Prompt Fonksiyonu (Eğitimdeki ile AYNI OLMALI) ---
def generate_inference_prompt(text_input):
    return f"""
Analyze the sentiment of the news headline enclosed in square brackets,
determine if it is positive, neutral, or negative, and return the answer as
the corresponding sentiment label "positive" or "neutral" or "negative".

[{text_input}] = """.strip()  # strip() fonksiyonu baştaki ve sondaki boşlukları temizler

# --- Test Edilecek Örnek Finansal Cümleler --- KENDİ CÜMLELERİNİZİ BURAYA GİREBİLİRSİNİZ..
sample_financial_texts = [
    "The company announced record profits for the third consecutive quarter, signaling strong growth.",
    "Market volatility and rising interest rates are expected to negatively impact Q4 earnings.",
    "The GDP figures released today were in line with economist projections for the period.",
    "Despite a challenging economic climate, the firm managed to secure new major contracts.",
    "Analysts predict a downturn in the housing market due to oversupply and decreasing demand."
]

print("Test için örnek cümleler ve prompt formatı hazır.")

Test için örnek cümleler ve prompt formatı hazır.


In [13]:
# --- Pipeline ile Tahmin Yapma ---
print("\\nTransformers Pipeline ile tahminler yapılıyor...")

try:
    sentiment_pipeline = pipeline(
        "text-generation",
        model=model_with_adapters,
        tokenizer=tokenizer
    )
    print("Pipeline başarıyla oluşturuldu.")

    print(f"\\n--- Test Sonuçları (Pipeline) ---")
    for text in sample_financial_texts:
        prompt = generate_inference_prompt(text)
        try:
            result = sentiment_pipeline(
                prompt,
                max_new_tokens=15,
                do_sample=False,
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id
            )
            generated_text_full = result[0]['generated_text']

            answer_part = ""
            if generated_text_full.startswith(prompt):
                answer_part = generated_text_full[len(prompt):]
            elif "=" in generated_text_full:
                answer_part = generated_text_full.split("=")[-1]
            else:
                words_in_output = generated_text_full.split()
                if words_in_output:
                    answer_part = words_in_output[-1]

            predicted_sentiment_raw = answer_part.strip().lower().replace("\"", "").replace(".", "")

            final_predicted_sentiment = "unknown"
            if predicted_sentiment_raw.startswith("positive"):
                final_predicted_sentiment = "positive"
            elif predicted_sentiment_raw.startswith("negative"):
                final_predicted_sentiment = "negative"
            elif predicted_sentiment_raw.startswith("neutral"):
                final_predicted_sentiment = "neutral"
            elif predicted_sentiment_raw in ["positive", "negative", "neutral"]:
                final_predicted_sentiment = predicted_sentiment_raw

            print(f"\\nCümle: '{text}'")
            print(f"Modelin Ham Çıktısı (sadece üretilen): '{answer_part.strip()}'")
            print(f"Ayrıştırılmış Tahmin: {final_predicted_sentiment}")
            print("-" * 40)

        except Exception as e_pipe_single:
            print(f"'{text}' cümlesi için pipeline hatası: {e_pipe_single}")
            print("-" * 40)

except Exception as e_pipeline_setup:
    print(f"Pipeline oluşturulurken veya tahmin sırasında genel HATA: {e_pipeline_setup}")
    print("Modelin ve tokenizer'ın doğru yüklendiğinden emin olun.")

Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'Glm4ForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoFo

\nTransformers Pipeline ile tahminler yapılıyor...
Pipeline başarıyla oluşturuldu.
\n--- Test Sonuçları (Pipeline) ---


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


\nCümle: 'The company announced record profits for the third consecutive quarter, signaling strong growth.'
Modelin Ham Çıktısı (sadece üretilen): 'positive
[The company's order book stood at 2.2'
Ayrıştırılmış Tahmin: positive
----------------------------------------
\nCümle: 'Market volatility and rising interest rates are expected to negatively impact Q4 earnings.'
Modelin Ham Çıktısı (sadece üretilen): 'negative
[The company's order book stood at 1.5'
Ayrıştırılmış Tahmin: negative
----------------------------------------
\nCümle: 'The GDP figures released today were in line with economist projections for the period.'
Modelin Ham Çıktısı (sadece üretilen): 'neutral

[The OMX Helsinki index was down 0.34 pct at'
Ayrıştırılmış Tahmin: neutral
----------------------------------------
\nCümle: 'Despite a challenging economic climate, the firm managed to secure new major contracts.'
Modelin Ham Çıktısı (sadece üretilen): 'positive
[The company's board of directors will propose a dividen